<a href="https://colab.research.google.com/github/PAURAB-BHATTACHARJEE/FAKE_NEWS_HINDI/blob/main/Hindi_Fake_News_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import string
# !pip install inltk
# from collections.abc import Iterable
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding,Bidirectional,Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split

In [ ]:
# !pip install --upgrade fastai
# !pip install --upgrade inltk

# !pip install abc
# from inltk.inltk import setup

In [ ]:
# from inltk.inltk import setup

In [ ]:

fake_csv=pd.read_csv("drive/MyDrive/FAKE_NEWS_DETECTION/hindi_fake.csv")
true_csv=pd.read_csv("drive/MyDrive/FAKE_NEWS_DETECTION/hindi_true.csv")

In [ ]:
fake_csv.head()

,text,label
0,इसके पहले एक तस्वीर वायरल है जो रोड पर 'गो बैक...,0
1,धातु के पहियों वाले ट्रैक्टर्स की पुरानी और अस...,0
2,सोशल मीडिया पर वायरल एक पोस्ट में दावा किया जा...,0
3,उत्तर प्रदेश (Uttar Pradesh) में आत्महत्या (su...,0
4,दिल्ली की सीमाओं पर केंद्र द्वारा लागू तीन कृष...,0


In [ ]:
true_csv.head()

,text,label
0,सात सालों तक एक दूसरे से नहीं की बातआपको जानकर...,1
1,"ममता बनर्जी की फोटो शेयर कर बोले प्रकाश राज, \...",1
2,"असम: कांग्रेस पर साधा राजनाथ सिंह ने निशाना, क...",1
3,सरवणन को टिकट मिलने से नाराजगीरविवार सुबह तिरु...,1
4,दिल'धक-धक गर्ल माधुरी दीक्षित के साथ आमिर ने स...,1


In [ ]:
true_csv.shape

(5177, 2)

In [ ]:
fake_csv.shape

(3481, 2)

In [ ]:
df=pd.concat([true_csv,fake_csv],axis=0)

In [ ]:
# Determine the size of the smaller dataset
min_size = min(len(true_csv), len(fake_csv))

# Undersample the larger dataset to match the size of the smaller dataset
df_true_undersampled = true_csv.sample(n=min_size, random_state=42)
df_fake_undersampled = fake_csv.sample(n=min_size, random_state=42)

# Concatenate the undersampled datasets
df_equal_size = pd.concat([df_true_undersampled, df_fake_undersampled], ignore_index=True)

# Shuffle the concatenated dataset
df_equal_size = df_equal_size.sample(frac=1, random_state=42).reset_index(drop=True)

# Now df_equal_size contains both datasets of equal size

In [ ]:
df.head()

,text,label
0,सात सालों तक एक दूसरे से नहीं की बातआपको जानकर...,1
1,"ममता बनर्जी की फोटो शेयर कर बोले प्रकाश राज, \...",1
2,"असम: कांग्रेस पर साधा राजनाथ सिंह ने निशाना, क...",1
3,सरवणन को टिकट मिलने से नाराजगीरविवार सुबह तिरु...,1
4,दिल'धक-धक गर्ल माधुरी दीक्षित के साथ आमिर ने स...,1


In [ ]:
df.tail()

,text,label
3476,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया पर...,0
3477,Shareविश्वास न्यूज (नई दिल्ली)। सोशल मीडिया पर...,0
3478,Shareनई दिल्ली (विश्वास न्यूज)। एक यूट्यूब चैन...,0
3479,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया पर...,0
3480,Shareनई दिल्‍ली (Vishvas News)। सोशल मीडिया मे...,0


In [ ]:
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)
df = pd.concat([true_csv, fake_csv], axis=0).drop_duplicates()


Empty DataFrame
Columns: [text, label]
Index: []


In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.head()

,text,label
2469,कांग्रेस के वरिष्ठ नेता अजय माकन की मानें तो ब...,0
3210,कोलकाता से लेकर दिल्ली तक मेट्रो के कर्णधार और...,1
145,कई दक्षिणपंथी ट्विटर हैंडल एवं फ़ेसबुक पेजों ने...,0
2951,हाथरस: सुनिए उस बेटी का दर्द जिसके पिता को गोल...,1
3149,नेशनल डेस्क.\nहिमाचल की बेटी आंचल ठाकुर वर्ल्ड...,1


In [ ]:
(df['label'] == 0).sum()


3481

In [ ]:
df

,text,label
2469,कांग्रेस के वरिष्ठ नेता अजय माकन की मानें तो ब...,0
3210,कोलकाता से लेकर दिल्ली तक मेट्रो के कर्णधार और...,1
145,कई दक्षिणपंथी ट्विटर हैंडल एवं फ़ेसबुक पेजों ने...,0
2951,हाथरस: सुनिए उस बेटी का दर्द जिसके पिता को गोल...,1
3149,नेशनल डेस्क.\nहिमाचल की बेटी आंचल ठाकुर वर्ल्ड...,1
...,...,...
1388,कई लोग कह रहे हैं कि ये साल 2020 में भारतीय नौ...,0
4663,बाटला हाउस एनकाउंटर पर सवाल उठाने वालों पर कें...,1
1431,बिहार के चुनावी माहौल के बीच सोशल मीडिया पर आर...,0
2193,उप भारतीय सेना प्रमुख लेफ्टिनेंट जनरल एस.के.\n...,1


In [ ]:
def wordopt(text):
    # text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
# from collections.abc import Iterable
# import inltk
# from inltk.inltk import setup
# from inltk import setup
# setup('hi')
# print(dir(inltk))



In [ ]:
# !pip show inltk

In [ ]:
# import inltk
# print(dir(inltk))


In [ ]:
# df_equal_size['text'] = df_equal_size['text'].apply(wordopt)
df['text'] = df['text'].apply(wordopt)


In [ ]:

X = df['text']
y= df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X = df_equal_size['text']
# y= df_equal_size['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
"hello world hi bro"

In [ ]:
# print(tfidf_matrix)

In [ ]:
# tfidf_matrix.shape

In [ ]:
# tfidf_matrix[0, 0]

In [ ]:
# tfidf_matrix[0, 18996]

In [ ]:
# Convert sparse matrices to dense arrays
X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()


In [ ]:
input_dim=X_train_dense.shape[1]
print(input_dim)
# X_train_dense.shape()


21960


In [ ]:
#LSTM/BiDiecrtinal LSTM: accuracy=56%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
 30/217 [===>..........................] - ETA: 3:24 - loss: 0.6851 - accuracy: 0.5688

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18326, 128)        2345728   
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 2609153 (9.95 MB)
Trainable params: 2609153 (9.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#CNN Model: accuracy=59%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
217/217 [==============================] - 51s 206ms/step - loss: 0.6765 - accuracy: 0.5989 - val_loss: 0.6783 - val_accuracy: 0.5883
Epoch 2/10
217/217 [==============================] - 45s 208ms/step - loss: 0.6740 - accuracy: 0.6003 - val_loss: 0.6775 - val_accuracy: 0.5883
Epoch 3/10
217/217 [==============================] - 43s 197ms/step - loss: 0.6738 - accuracy: 0.6003 - val_loss: 0.6775 - val_accuracy: 0.5883
Epoch 4/10
217/217 [==============================] - 45s 207ms/step - loss: 0.6733 - accuracy: 0.6003 - val_loss: 0.6783 - val_accuracy: 0.5883
Epoch 5/10
196/217 [==========================>...] - ETA: 3s - loss: 0.6736 - accuracy: 0.5993

KeyboardInterrupt: 

In [ ]:
## cnn + lstm  : accuracy=59%

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=128, input_length=X_train_dense.shape[1]))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=128))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_dense, y_train, epochs=10, batch_size=32, validation_data=(X_test_dense, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
 31/217 [===>..........................] - ETA: 1:11 - loss: 0.6784 - accuracy: 0.5988

KeyboardInterrupt: 

In [ ]:
!pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227138 sha256=c32714c57480c94a792bb5d24acd5a85c2b816c4aa4f0730ad706940ef928dba
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
# import fasttext

In [ ]:
# # Split the combined DataFrame into training and testing sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# # Separate the text and label data for training and testing
# X_train = train_df['text']
# y_train = train_df['label']
# X_test = test_df['text']
# y_test = test_df['label']

In [ ]:
# print("Data type of y_train:", type(y_train))
# y_train_list = y_train.tolist()


Data type of y_train: <class 'pandas.core.series.Series'>


In [ ]:
print(len(y_train))

6926


In [ ]:
print(len(X_train_dense))

6926


In [ ]:
# FASTTEXT MODEL : accuracy=93%
import fasttext



def wordopt(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

df['text'] = df['text'].apply(wordopt)


X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  labels are converted to the FastText format
def format_label(label):
    return '__label__' + str(label)

# Create training data with labels in FastText format
y_train_list = y_train.tolist()
formatted_labels_train = [format_label(label) for label in y_train_list]

# Write the training data to a file
with open('train.txt', 'w') as f:
    for text, label in zip(X_train, formatted_labels_train):
        f.write(label + ' ' + text + '\n')

# Created test data with labels in FastText format
y_test_list = y_test.tolist()
formatted_labels_test = [format_label(label) for label in y_test_list]

# Write the test data to a file
with open('test.txt', 'w') as f:
    for text, label in zip(X_test, formatted_labels_test):
        f.write(label + ' ' + text + '\n')

# Train the FastText model
model = fasttext.train_supervised(input='train.txt')

# Evaluate the model using FastText's built-in method
test_result = model.test('test.txt')
print(f'Number of examples: {test_result[0]}')
print(f'Precision: {test_result[1]}')
print(f'Recall: {test_result[2]}')

Number of examples: 1732
Precision: 0.9295612009237876
Recall: 0.9295612009237876


In [ ]:
# mBERT model : accuracy=59%

In [ ]:
pip install transformers datasets tensorflow


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import string
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import Dataset
import tensorflow as tf

# Preprocess text function
def wordopt(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


# Apply preprocessing to the text column
df['text'] = df['text'].apply(wordopt)

# Split the data into training and testing sets
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to a Hugging Face Dataset
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test}))

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)  # Truncate to 128 tokens

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for TensorFlow
train_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])

# Convert datasets to tf.data.Dataset
train_features = {x: train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_dataset["label"])).shuffle(100).batch(16)  # Reduced batch size

test_features = {x: test_dataset[x] for x in tokenizer.model_input_names}
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_dataset["label"])).batch(16)  # Reduced batch size

# Load the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Explicitly define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# # Compile the model with TensorFlow's Keras API
# model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

# # Train the model
# model.fit(train_tf_dataset, epochs=3, validation_data=test_tf_dataset)

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(train_tf_dataset, epochs=10, batch_size=32, validation_data=(test_tf_dataset))

# Evaluate the model
loss, accuracy = model.evaluate(test_tf_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/6926 [00:00<?, ? examples/s]

Map:   0%|          | 0/1732 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
433/433 [==============================] - 283s 482ms/step - loss: 6.4683 - accuracy: 0.5741 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 2/10
433/433 [==============================] - 196s 454ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 3/10
433/433 [==============================] - 196s 452ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 4/10
433/433 [==============================] - 196s 453ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 5/10
433/433 [==============================] - 197s 454ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 6/10
433/433 [==============================] - 196s 452ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 - val_accuracy: 0.5883
Epoch 7/10
433/433 [==============================] - 196s 453ms/step - loss: 6.0944 - accuracy: 0.6003 - val_loss: 6.2775 -

In [ ]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

# Load the MURIL tokenizer and model
model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
muril_model = TFAutoModel.from_pretrained(model_name)



# Convert target variables to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

def tokenize_and_encode(texts, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',  # Ensure padding to max_length
            truncation=True,       # Ensure truncation to max_length
            return_attention_mask=True,
            return_tensors='tf'
        )

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

# Tokenize and encode the training data
X_train_encodings, X_train_masks = tokenize_and_encode(X_train, tokenizer)
X_test_encodings, X_test_masks = tokenize_and_encode(X_test, tokenizer)

# Extract embeddings from MURIL
def extract_embeddings(bert_model, input_ids, attention_masks):
    outputs = bert_model(input_ids, attention_mask=attention_masks)
    embeddings = outputs.last_hidden_state
    return embeddings

# Define a function to process in smaller batches
def batch_process(func, bert_model, input_ids, attention_masks, batch_size=2):
    results = []
    for i in range(0, input_ids.shape[0], batch_size):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_attention_masks = attention_masks[i:i + batch_size]
        result = func(bert_model, batch_input_ids, batch_attention_masks)
        results.append(result)
    return tf.concat(results, axis=0)

X_train_embeddings = batch_process(extract_embeddings, muril_model, X_train_encodings, X_train_masks)
X_test_embeddings = batch_process(extract_embeddings, muril_model, X_test_encodings, X_test_masks)

# Free up memory by deleting the input encodings and masks
del X_train_encodings, X_train_masks, X_test_encodings, X_test_masks

# Define your CNN/LSTM/BiLSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train_embeddings.shape[1], X_train_embeddings.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.LSTM(units=128))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile model with mixed precision policy
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model with reduced batch size
model.fit(X_train_embeddings, y_train, epochs=10, batch_size=2, validation_data=(X_test_embeddings, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test_embeddings, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some layers from the model checkpoint at google/muril-base-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
3463/3463 [==============================] - 231s 65ms/step - loss: 0.6126 - accuracy: 0.6655 - val_loss: 0.4646 - val_accuracy: 0.7829
Epoch 2/10
3463/3463 [==============================] - 233s 67ms/step - loss: 0.4458 - accuracy: 0.8058 - val_loss: 0.3438 - val_accuracy: 0.8730
Epoch 3/10
3463/3463 [==============================] - 229s 66ms/step - loss: 0.3800 - accuracy: 0.8500 - val_loss: 0.3706 - val_accuracy: 0.8441
Epoch 4/10
3463/3463 [==============================] - 231s 67ms/step - loss: 0.3695 - accuracy: 0.8597 - val_loss: 0.3200 - val_accuracy: 0.8828
Epoch 5/10
3463/3463 [==============================] - 228s 66ms/step - loss: 0.3890 - accuracy: 0.8428 - val_loss: 0.3887 - val_accuracy: 0.8406
Epoch 6/10
3463/3463 [==============================] - 227s 66ms/step - loss: 0.4299 - accuracy: 0.8236 - val_loss: 0.3159 - val_accuracy: 0.8926
Epoch 7/10
3463/3463 [==============================] - 234s 68ms/step - loss: 0.3310 - accuracy: 0.8698 - val_loss: 0